import libraries

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from utils import load_json, load_pickle

Settings

In [ ]:
baseline_folder = "../FP_prediction/baseline_models/models_cached/"
MIST_folder = "../FP_prediction/mist/models_cached/"

Helper Functions

In [ ]:
def load_json(path):

    with open(path, "r") as f:

        return json.load(f)

Consolidate results

In [ ]:
# Manually add in (hack)

all_folders = []
all_test_performances = {"loss": {}, "jaccard": {}} 

for folder in [MIST_folder, baseline_folder]:

    for model_type in os.listdir(folder):
        subfolder = os.path.join(folder, model_type)
        for dataset in os.listdir(subfolder):
            subsubfolder = os.path.join(subfolder, dataset)
            for checkpoint in os.listdir(subsubfolder):
                all_folders.append(os.path.join(subsubfolder, checkpoint))

for f in all_folders:

    try:
        expt_name = f.split("/")[-1]
        test_performance = load_json(os.path.join(f, "test_performance.json"))
        for k, v in test_performance.items():
            all_test_performances[k][expt_name] = round(v, 5)
    except Exception as e:
        print(e)
        continue

Format the table

In [ ]:
results = pd.DataFrame.from_dict(all_test_performances)
results = results.reset_index()
results = results.rename(columns = {"index": "expt_name"})
results.loc[:, "dataset"] = results.loc[:, "expt_name"].apply(lambda x: x.split("_")[0]) 
results.loc[:, "meta"] = results.loc[:, "expt_name"].apply(lambda x: "meta" in x) 
results.loc[:, "model"] = results.loc[:, "expt_name"].apply(lambda x: x.split("_")[1]) 
results.loc[:, "split"] = results.loc[:, "expt_name"].apply(lambda x: "_".join(x.split("_")[-2:]).replace("4096_", "")) 

results = results.loc[:, ["dataset", "model", "split", "meta", "loss", "jaccard"]]

results = results.sort_values(by = 'split', ascending = False)
results = results.sort_values(by = 'model', ascending = False)
results = results.sort_values(by = 'dataset', ascending = False)

Seperate results into w and wo meta

In [ ]:
results_wo_meta = results.loc[results.loc[:, "meta"].apply(lambda x: x is False), :]
results_w_meta = results.loc[results.loc[:, "meta"].apply(lambda x: x is True), :]

In [ ]:
for dataset in ["NIST2023", "MSG", "C"]:
    for split in ["random", "scaffold_vanilla", "inchikey_vanilla"]:

        sub_results_wo_meta = results_wo_meta.loc[results_wo_meta.apply(lambda x : x["dataset"] == dataset and x["split"] == split, axis = 1), :]
        print(sub_results_wo_meta)
        print("*" * 80)
        print() 

In [ ]:
for dataset in ["NIST2023", "MSG", "C"]:
    for split in ["random", "scaffold_vanilla", "inchikey_vanilla"]:

        sub_results_w_meta = results_w_meta.loc[results_w_meta.apply(lambda x : x["dataset"] == dataset and x["split"] == split, axis = 1), :]
        print(sub_results_w_meta)
        print("*" * 80)
        print() 